In [21]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [22]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [23]:
tf.keras.backend.clear_session()

In [24]:
subject = 'Goiás - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [25]:
data = pd.read_csv('2003_mo_model_input_GO.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Goiás - Produção de Cimento (t),Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Goiais-03anhão - value,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - Desemprego,Goiás - Consumo de Cimento (t)
0,2003-1,56.459960,0.745293,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.260097,9.371822e+07,5.518325e+06,14.031270,8.418248e+07,8.179785,97.447
1,2003-2,56.789152,0.745405,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.258964,9.383590e+07,5.523290e+06,14.034476,8.422082e+07,8.174086,97.256
2,2003-3,57.140263,0.745517,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.257352,9.395358e+07,5.528254e+06,14.037682,8.425916e+07,8.168386,101.442
3,2003-4,57.649192,0.745629,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.255929,9.407126e+07,5.533219e+06,14.040888,8.429750e+07,8.162687,98.217
4,2003-5,58.213295,0.745741,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.254109,9.418894e+07,5.538184e+06,14.044093,8.433584e+07,8.156988,112.540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,175.674487,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.520178,NaN,NaN,NaN,NaN,NaN,274.673
236,2022-9,174.614843,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.518515,NaN,NaN,NaN,NaN,NaN,247.240
237,2022-10,173.307223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.516447,NaN,NaN,NaN,NaN,NaN,246.647
238,2022-11,171.556538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.513912,NaN,NaN,NaN,NaN,NaN,225.813


In [26]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Goiás - Produção de Cimento (t),Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Goiais-03anhão - value,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - Desemprego
0,-1.656649,-2.298583,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.950106,-1.718818,-1.724379,-2.510500,-2.091868,-0.400818
1,-1.644539,-2.259040,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.970613,-1.700304,-1.668988,-2.459954,-2.059753,-0.411374
2,-1.631624,-2.219497,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.999751,-1.681790,-1.613597,-2.409407,-2.027637,-0.421930
3,-1.612902,-2.179955,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.025491,-1.663277,-1.558206,-2.358860,-1.995522,-0.432487
4,-1.592151,-2.140412,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.058403,-1.644763,-1.502815,-2.308314,-1.963406,-0.443043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.639330,0.726172,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.221276,1.118822,-1.173185,-0.446937,0.837128,0.999503
188,1.677410,0.682578,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.236987,1.111624,-1.156891,-0.447051,0.832077,1.003711
189,1.732332,0.638983,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.252847,1.104427,-1.140596,-0.447166,0.827027,1.007918
190,1.763870,0.595389,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.276241,1.097229,-1.124301,-0.447280,0.821977,1.012126


In [27]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0       93.660
1       85.349
2      116.097
3      120.197
4      122.728
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: Goiás - Consumo de Cimento (t), Length: 240, dtype: float64

In [28]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Goiás - Produção de Cimento (t),Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Goiais-03anhão - value,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - Desemprego
0,-1.656649,-2.298583,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.950106,-1.718818,-1.724379,-2.510500,-2.091868,-0.400818
1,-1.644539,-2.259040,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.970613,-1.700304,-1.668988,-2.459954,-2.059753,-0.411374
2,-1.631624,-2.219497,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.999751,-1.681790,-1.613597,-2.409407,-2.027637,-0.421930
3,-1.612902,-2.179955,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.025491,-1.663277,-1.558206,-2.358860,-1.995522,-0.432487
4,-1.592151,-2.140412,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.058403,-1.644763,-1.502815,-2.308314,-1.963406,-0.443043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.623245,1.561651,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,0.944351,1.175698,-1.121085,-0.108013,0.955652,1.757990
158,0.644558,1.540971,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,0.952621,1.178649,-1.137848,-0.116498,0.956673,1.696610
159,0.663382,1.520292,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,0.960982,1.181601,-1.154611,-0.124984,0.957693,1.635230
160,0.670170,1.499612,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,0.969363,1.184552,-1.171374,-0.133469,0.958714,1.573850


In [29]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0       93.660
1       85.349
2      116.097
3      120.197
4      122.728
        ...   
157    146.310
158    191.374
159    164.318
160    211.049
161    198.789
Name: Goiás - Consumo de Cimento (t), Length: 162, dtype: float64

In [30]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [31]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [32]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Goiás - Produção de Cimento (t),Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Goiais-03anhão - value,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - Desemprego
126,0.798828,0.605338,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.673675,0.830615,0.695340,0.936426,0.971320,0.568035
127,0.774031,0.608897,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.687412,0.848214,0.652086,0.905979,0.976868,0.664626
128,0.731137,0.612457,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.706891,0.865813,0.608833,0.875531,0.982416,0.761218
129,0.695886,0.616016,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.722784,0.883412,0.565579,0.845084,0.987963,0.857809
130,0.655420,0.619576,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.738712,0.901011,0.522326,0.814636,0.993511,0.954400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.639330,0.726172,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.221276,1.118822,-1.173185,-0.446937,0.837128,0.999503
188,1.677410,0.682578,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.236987,1.111624,-1.156891,-0.447051,0.832077,1.003711
189,1.732332,0.638983,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.252847,1.104427,-1.140596,-0.447166,0.827027,1.007918
190,1.763870,0.595389,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.276241,1.097229,-1.124301,-0.447280,0.821977,1.012126


In [33]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [34]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [35]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [36]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [37]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3303266240, 1499702046, 539933062, 2661099862, 2142438238, 1634101702, 1832282949, 755849656, 2612672804, 3742698112, 3182054575, 1356891045, 2734674561, 3972373667, 3946624559, 2770861280, 1773878620, 3908927384, 1656611875, 1963917445, 2293679106, 2182852254, 2322502778, 1003883322, 878844930, 3268884385, 716320544, 3387672145, 829771409, 1029183192, 3202176650, 1624168554, 2320703813, 634714680, 2737147616, 4199967476, 2985857992, 2489030224, 2451376091, 876237196, 949353239, 1298857912, 2777123247, 2577037386, 920082508, 2761247419, 3696981203, 1447470684, 4013323476, 940090059]


Step: 0 ___________________________________________
val_loss: 373.62310791015625
winner_seed: 3303266240


Step: 1 ___________________________________________
val_loss: 268.6561279296875
winner_seed: 1499702046


Step: 2 ___________________________________________
val_loss: 926.1270751953125


Step: 3 ___________________________________________
val_loss: 297.05084228515625


Step: 4 _____________________

In [38]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 113ms/step - loss: 37886.3789 - val_loss: 2471.0635
Epoch 2/10000
4/4 [==============================] - 0s 8ms/step - loss: 2214.6653 - val_loss: 1769.9276
Epoch 3/10000
4/4 [==============================] - 0s 8ms/step - loss: 1904.3950 - val_loss: 2960.7131
Epoch 4/10000
4/4 [==============================] - 0s 8ms/step - loss: 2567.5674 - val_loss: 1815.7505
Epoch 5/10000
4/4 [==============================] - 0s 9ms/step - loss: 1911.5178 - val_loss: 2051.5674
Epoch 6/10000
4/4 [==============================] - 0s 9ms/step - loss: 2016.1844 - val_loss: 1736.6901
Epoch 7/10000
4/4 [==============================] - 0s 8ms/step - loss: 2150.3176 - val_loss: 1673.7220
Epoch 8/10000
4/4 [==============================] - 0s 9ms/step - loss: 1938.9375 - val_loss: 1793.3398
Epoch 9/10000
4/4 [==============================] - 0s 9ms/step - loss: 1852.8442 - val_loss: 2127.5183
Epoch 10/10000
4/4 [==============================] 

4/4 [==============================] - 0s 9ms/step - loss: 606.3531 - val_loss: 1266.4231
Epoch 80/10000
4/4 [==============================] - 0s 9ms/step - loss: 864.4655 - val_loss: 1192.7953
Epoch 81/10000
4/4 [==============================] - 0s 9ms/step - loss: 831.0706 - val_loss: 765.3505
Epoch 82/10000
4/4 [==============================] - 0s 9ms/step - loss: 526.7811 - val_loss: 835.6422
Epoch 83/10000
4/4 [==============================] - 0s 9ms/step - loss: 502.5466 - val_loss: 756.6990
Epoch 84/10000
4/4 [==============================] - 0s 9ms/step - loss: 594.0535 - val_loss: 745.8979
Epoch 85/10000
4/4 [==============================] - 0s 9ms/step - loss: 582.8315 - val_loss: 768.5273
Epoch 86/10000
4/4 [==============================] - 0s 9ms/step - loss: 516.3749 - val_loss: 1031.3276
Epoch 87/10000
4/4 [==============================] - 0s 10ms/step - loss: 645.1114 - val_loss: 716.4620
Epoch 88/10000
4/4 [==============================] - 0s 10ms/step - loss: 

4/4 [==============================] - 0s 9ms/step - loss: 484.4019 - val_loss: 731.7880
Epoch 158/10000
4/4 [==============================] - 0s 9ms/step - loss: 608.5377 - val_loss: 626.1257
Epoch 159/10000
4/4 [==============================] - 0s 8ms/step - loss: 445.0903 - val_loss: 670.2878
Epoch 160/10000
4/4 [==============================] - 0s 9ms/step - loss: 427.7416 - val_loss: 645.8702
Epoch 161/10000
4/4 [==============================] - 0s 9ms/step - loss: 469.1571 - val_loss: 681.9308
Epoch 162/10000
4/4 [==============================] - 0s 9ms/step - loss: 573.4899 - val_loss: 641.2495
Epoch 163/10000
4/4 [==============================] - 0s 9ms/step - loss: 563.9066 - val_loss: 622.9365
Epoch 164/10000
4/4 [==============================] - 0s 9ms/step - loss: 419.6928 - val_loss: 667.6382
Epoch 165/10000
4/4 [==============================] - 0s 9ms/step - loss: 483.4131 - val_loss: 613.6288
Epoch 166/10000
4/4 [==============================] - 0s 9ms/step - lo

4/4 [==============================] - 0s 9ms/step - loss: 541.8175 - val_loss: 600.7515
Epoch 236/10000
4/4 [==============================] - 0s 9ms/step - loss: 525.8255 - val_loss: 690.0908
Epoch 237/10000
4/4 [==============================] - 0s 9ms/step - loss: 542.0021 - val_loss: 630.2566
Epoch 238/10000
4/4 [==============================] - 0s 9ms/step - loss: 675.1554 - val_loss: 623.6121
Epoch 239/10000
4/4 [==============================] - 0s 9ms/step - loss: 456.2342 - val_loss: 681.4497
Epoch 240/10000
4/4 [==============================] - 0s 10ms/step - loss: 443.8521 - val_loss: 549.3718
Epoch 241/10000
4/4 [==============================] - 0s 10ms/step - loss: 477.4819 - val_loss: 791.9293
Epoch 242/10000
4/4 [==============================] - 0s 9ms/step - loss: 451.6099 - val_loss: 582.9739
Epoch 243/10000
4/4 [==============================] - 0s 9ms/step - loss: 431.7266 - val_loss: 629.1718
Epoch 244/10000
4/4 [==============================] - 0s 9ms/step - 

4/4 [==============================] - 0s 9ms/step - loss: 278.0034 - val_loss: 655.9246
Epoch 314/10000
4/4 [==============================] - 0s 9ms/step - loss: 457.5110 - val_loss: 662.5494
Epoch 315/10000
4/4 [==============================] - 0s 8ms/step - loss: 343.3441 - val_loss: 625.0054
Epoch 316/10000
4/4 [==============================] - 0s 8ms/step - loss: 269.0681 - val_loss: 680.7182
Epoch 317/10000
4/4 [==============================] - 0s 7ms/step - loss: 288.4101 - val_loss: 828.0840
Epoch 318/10000
4/4 [==============================] - 0s 7ms/step - loss: 296.2287 - val_loss: 695.4642
Epoch 319/10000
4/4 [==============================] - 0s 7ms/step - loss: 269.3547 - val_loss: 870.0474
Epoch 320/10000
4/4 [==============================] - 0s 8ms/step - loss: 648.2716 - val_loss: 947.9611
Epoch 321/10000
4/4 [==============================] - 0s 7ms/step - loss: 412.1292 - val_loss: 804.3279
Epoch 322/10000
4/4 [==============================] - 0s 7ms/step - lo

...

4/4 [==============================] - 0s 9ms/step - loss: 170.3386 - val_loss: 361.9031
Epoch 1324/10000
4/4 [==============================] - 0s 9ms/step - loss: 102.6971 - val_loss: 360.7976
Epoch 1325/10000
4/4 [==============================] - 0s 9ms/step - loss: 116.4343 - val_loss: 307.4381
Epoch 1326/10000
4/4 [==============================] - 0s 9ms/step - loss: 106.5354 - val_loss: 347.5289
Epoch 1327/10000
4/4 [==============================] - 0s 9ms/step - loss: 154.8729 - val_loss: 301.7280
Epoch 1328/10000
4/4 [==============================] - 0s 9ms/step - loss: 136.5980 - val_loss: 337.6290
Epoch 1329/10000
4/4 [==============================] - 0s 9ms/step - loss: 96.9820 - val_loss: 392.7325
Epoch 1330/10000
4/4 [==============================] - 0s 9ms/step - loss: 96.1830 - val_loss: 443.0040
Epoch 1331/10000
4/4 [==============================] - 0s 10ms/step - loss: 94.8490 - val_loss: 400.3271
Epoch 1332/10000
4/4 [==============================] - 0s 9ms/st

4/4 [==============================] - 0s 9ms/step - loss: 85.2261 - val_loss: 417.1770
Epoch 1401/10000
4/4 [==============================] - 0s 9ms/step - loss: 104.7780 - val_loss: 358.9694
Epoch 1402/10000
4/4 [==============================] - 0s 8ms/step - loss: 95.6764 - val_loss: 428.9338
Epoch 1403/10000
4/4 [==============================] - 0s 9ms/step - loss: 155.0477 - val_loss: 353.8129
Epoch 1404/10000
4/4 [==============================] - 0s 9ms/step - loss: 88.6724 - val_loss: 446.6189
Epoch 1405/10000
4/4 [==============================] - 0s 9ms/step - loss: 98.5964 - val_loss: 473.0118
Epoch 1406/10000
4/4 [==============================] - 0s 9ms/step - loss: 121.0236 - val_loss: 604.8519
Epoch 1407/10000
4/4 [==============================] - 0s 9ms/step - loss: 103.4097 - val_loss: 423.5262
Epoch 1408/10000
4/4 [==============================] - 0s 10ms/step - loss: 95.2942 - val_loss: 338.8640
Epoch 1409/10000
4/4 [==============================] - 0s 9ms/step

Epoch 1478/10000
4/4 [==============================] - 0s 9ms/step - loss: 71.2106 - val_loss: 548.5740
Epoch 1479/10000
4/4 [==============================] - 0s 8ms/step - loss: 102.2986 - val_loss: 285.5791
Epoch 1480/10000
4/4 [==============================] - 0s 8ms/step - loss: 75.1099 - val_loss: 329.4167
Epoch 1481/10000
4/4 [==============================] - 0s 8ms/step - loss: 95.5154 - val_loss: 347.7003
Epoch 1482/10000
4/4 [==============================] - 0s 9ms/step - loss: 121.7589 - val_loss: 364.4897
Epoch 1483/10000
4/4 [==============================] - 0s 10ms/step - loss: 79.5568 - val_loss: 453.4982
Epoch 1484/10000
4/4 [==============================] - 0s 9ms/step - loss: 111.5885 - val_loss: 487.0251
Epoch 1485/10000
4/4 [==============================] - 0s 9ms/step - loss: 111.3173 - val_loss: 485.5412
Epoch 1486/10000
4/4 [==============================] - 0s 9ms/step - loss: 94.7139 - val_loss: 393.1891
Epoch 1487/10000
4/4 [============================

4/4 [==============================] - 0s 8ms/step - loss: 99.8380 - val_loss: 393.6383
Epoch 1556/10000
4/4 [==============================] - 0s 8ms/step - loss: 83.2902 - val_loss: 362.7285
Epoch 1557/10000
4/4 [==============================] - 0s 9ms/step - loss: 95.3264 - val_loss: 412.4521
Epoch 1558/10000
4/4 [==============================] - 0s 9ms/step - loss: 87.5853 - val_loss: 532.6595
Epoch 1559/10000
4/4 [==============================] - 0s 9ms/step - loss: 102.1157 - val_loss: 543.9612
Epoch 1560/10000
4/4 [==============================] - 0s 8ms/step - loss: 65.2999 - val_loss: 457.3532
Epoch 1561/10000
4/4 [==============================] - 0s 9ms/step - loss: 101.6567 - val_loss: 524.0873
Epoch 1562/10000
4/4 [==============================] - 0s 9ms/step - loss: 81.3509 - val_loss: 525.7743
Epoch 1563/10000
4/4 [==============================] - 0s 9ms/step - loss: 103.3910 - val_loss: 557.7106
Epoch 1564/10000
4/4 [==============================] - 0s 9ms/step -

Epoch 1633/10000
4/4 [==============================] - 0s 9ms/step - loss: 75.6032 - val_loss: 270.2899
Epoch 1634/10000
4/4 [==============================] - 0s 9ms/step - loss: 90.6444 - val_loss: 304.4336
Epoch 1635/10000
4/4 [==============================] - 0s 9ms/step - loss: 82.2663 - val_loss: 326.7955
Epoch 1636/10000
4/4 [==============================] - 0s 9ms/step - loss: 95.7591 - val_loss: 274.7619
Epoch 1637/10000
4/4 [==============================] - 0s 9ms/step - loss: 88.7877 - val_loss: 306.4193
Epoch 1638/10000
4/4 [==============================] - 0s 9ms/step - loss: 93.6672 - val_loss: 296.1623
Epoch 1639/10000
4/4 [==============================] - 0s 10ms/step - loss: 74.2175 - val_loss: 308.2315
Epoch 1640/10000
4/4 [==============================] - 0s 9ms/step - loss: 70.8170 - val_loss: 330.1447
Epoch 1641/10000
4/4 [==============================] - 0s 10ms/step - loss: 90.9942 - val_loss: 326.1945
Epoch 1642/10000
4/4 [==============================]

In [39]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [40]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,191.955505,193.299759,206.341949,202.07428,198.398743,193.064011,189.741547,183.143951,182.971008,188.657486,189.338364,186.591736,184.104919,185.419281,182.766159,181.180679,182.174652,179.270111,171.824844,173.379135,165.60611,163.995575,169.797684,167.344055,174.764618,165.11911,167.670761,164.098068,165.335724,164.129242
Target,205.551,230.558,209.621,210.984,189.889,143.721,176.275,152.632,177.876,181.384,160.12,236.951,206.557,224.27,213.871,211.066,182.899,159.002,178.918,161.936,173.792,176.627,207.991,185.784,234.176,226.996,218.141,234.939,199.619,168.797
Error,13.59549,37.25824,3.279053,8.909714,8.509735,49.343018,13.466553,30.511948,5.095001,7.273483,29.218369,50.359268,22.452087,38.850723,31.104843,29.885315,0.72435,20.268112,7.093155,11.44313,8.185898,12.631424,38.193314,18.439941,59.411377,61.876892,50.470245,70.840927,34.283279,4.667755


In [41]:
display(mae)
display(mape)

25.921423

0.12948634

In [42]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [43]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

NameError: name 'test_target' is not defined